## 0.0 read data

In [1]:
import numpy as np
from numpy import linalg
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import time
from datetime import *
import copy

In [137]:
df_origin = pd.read_csv('./data/train.csv')
label = pd.read_csv('./data/train_label.csv')
df_test = pd.read_csv('./data/test.csv')

In [138]:
df_origin = df_origin.drop(columns=['reservation_status'])

In [139]:
# convert month name to values
m = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
df_origin.arrival_date_month = df_origin.arrival_date_month.map(m)
df_test.arrival_date_month = df_test.arrival_date_month.map(m)

In [140]:
# convert hotel type to binary
h = {'City Hotel':0, 'Resort Hotel':1}
df_origin.hotel = df_origin.hotel.map(h)
df_test.hotel = df_test.hotel.map(h)

In [141]:
df_origin

,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date
0,0,1,0,342,2015,7,27,1,0,0,...,3,No Deposit,NaN,NaN,0,Transient,-6.305161,0,0,2015-07-01
1,1,0,0,257,2015,7,27,1,0,2,...,1,No Deposit,6.0,NaN,0,Transient,75.052227,0,0,2015-07-03
2,2,0,0,257,2015,7,27,1,0,2,...,0,No Deposit,6.0,NaN,0,Transient,74.546401,0,0,2015-07-03
3,3,0,0,257,2015,7,27,1,0,2,...,0,No Deposit,6.0,NaN,0,Transient,76.376288,0,0,2015-07-03
4,4,0,0,257,2015,7,27,1,0,2,...,0,No Deposit,6.0,NaN,0,Transient,49.411647,0,0,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,91526,1,1,19,2017,3,13,31,0,2,...,0,No Deposit,250.0,NaN,0,Transient,79.223571,0,1,2017-03-22
91527,91527,1,0,28,2017,3,13,31,0,2,...,0,No Deposit,NaN,NaN,0,Transient,-6.822102,0,0,2017-04-02
91528,91528,1,0,2,2017,3,13,31,0,1,...,1,No Deposit,NaN,NaN,0,Transient,90.814554,0,2,2017-04-01
91529,91529,1,0,30,2017,3,13,31,3,7,...,0,No Deposit,250.0,NaN,0,Transient,38.135565,0,1,2017-04-10


In [142]:
df_test

,ID,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests
0,91531,0,75,2017,4,13,1,2,5,2,...,D,D,0,No Deposit,9.0,NaN,0,Transient,0,1
1,91532,0,208,2017,4,13,1,4,10,2,...,A,A,0,No Deposit,9.0,NaN,0,Transient,0,1
2,91533,1,12,2017,4,13,1,2,5,2,...,A,A,0,No Deposit,40.0,NaN,0,Contract,0,1
3,91534,0,76,2017,4,13,1,2,5,3,...,D,D,0,No Deposit,9.0,NaN,0,Transient,0,0
4,91535,0,9,2017,4,13,1,2,4,2,...,F,F,0,No Deposit,9.0,NaN,0,Transient,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27854,119385,1,108,2017,8,35,31,2,5,2,...,E,E,0,No Deposit,241.0,NaN,0,Transient,0,1
27855,119386,1,194,2017,8,35,31,2,5,2,...,G,G,3,No Deposit,240.0,NaN,0,Transient,1,1
27856,119387,1,17,2017,8,35,31,0,3,2,...,A,A,0,No Deposit,240.0,NaN,0,Transient,0,2
27857,119388,1,191,2017,8,35,31,2,5,2,...,D,D,0,No Deposit,40.0,NaN,0,Contract,0,0


In [143]:
df_2016 = df_origin[df_origin['arrival_date_year'] == 2016]
df_201611 = df_2016[df_2016['arrival_date_month'] == 11]
df_20161130 = df_201611[df_201611['arrival_date_day_of_month'] == 30]

In [144]:
df_20161130

,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date
74776,74776,0,0,43,2016,11,49,30,2,4,...,0,No Deposit,9.0,NaN,0,Transient,89.492019,0,1,2016-12-06
74777,74777,0,0,42,2016,11,49,30,2,4,...,0,No Deposit,7.0,NaN,0,Transient,49.724337,0,1,2016-12-06
74778,74778,0,1,215,2016,11,49,30,0,4,...,0,No Deposit,9.0,NaN,0,Transient,42.398289,0,0,2016-08-03
74779,74779,0,0,15,2016,11,49,30,1,4,...,0,No Deposit,NaN,NaN,0,Transient,89.579433,0,0,2016-12-05
74780,74780,0,1,215,2016,11,49,30,0,4,...,1,No Deposit,9.0,NaN,0,Transient,37.282016,0,0,2016-08-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74838,74838,1,0,1,2016,11,49,30,0,3,...,0,No Deposit,240.0,NaN,0,Transient,56.939053,1,2,2016-12-03
74839,74839,1,0,1,2016,11,49,30,0,2,...,0,No Deposit,240.0,NaN,0,Transient,34.773759,1,1,2016-12-02
74840,74840,0,0,138,2016,11,49,30,0,3,...,3,No Deposit,9.0,NaN,0,Transient,72.291243,0,2,2016-12-03
74841,74841,0,0,12,2016,11,49,30,0,3,...,0,No Deposit,9.0,NaN,0,Transient,96.730231,0,2,2016-12-03


Split data at index 74843

:74842 -> train

74843: -> val

In [233]:
df_origin

,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date
0,0,1,0,342,2015,7,27,1,0,0,...,3,No Deposit,NaN,NaN,0,Transient,-6.305161,0,0,2015-07-01
1,1,0,0,257,2015,7,27,1,0,2,...,1,No Deposit,6.0,NaN,0,Transient,75.052227,0,0,2015-07-03
2,2,0,0,257,2015,7,27,1,0,2,...,0,No Deposit,6.0,NaN,0,Transient,74.546401,0,0,2015-07-03
3,3,0,0,257,2015,7,27,1,0,2,...,0,No Deposit,6.0,NaN,0,Transient,76.376288,0,0,2015-07-03
4,4,0,0,257,2015,7,27,1,0,2,...,0,No Deposit,6.0,NaN,0,Transient,49.411647,0,0,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,91526,1,1,19,2017,3,13,31,0,2,...,0,No Deposit,250.0,NaN,0,Transient,79.223571,0,1,2017-03-22
91527,91527,1,0,28,2017,3,13,31,0,2,...,0,No Deposit,NaN,NaN,0,Transient,-6.822102,0,0,2017-04-02
91528,91528,1,0,2,2017,3,13,31,0,1,...,1,No Deposit,NaN,NaN,0,Transient,90.814554,0,2,2017-04-01
91529,91529,1,0,30,2017,3,13,31,3,7,...,0,No Deposit,250.0,NaN,0,Transient,38.135565,0,1,2017-04-10


In [234]:
df_origin.dtypes 

ID                                  int64
hotel                               int64
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                  int64
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                      

In [249]:
df_tree = df_origin.fillna(df.children.median())
df_tree = df_tree.drop(columns = ['country', 'agent', 'company', 'reservation_status_date'])

In [250]:
df_tree.dtypes 

ID                                  int64
hotel                               int64
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                  int64
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
days_in_waiting_list              

In [251]:
# meal, market_segment, distribution_channel, reserved_room_type, assigned_room_type, deposit_type, customer_type
df_tree['meal'] = df_tree['meal'].map({'BB':1, 'FB': 2, 'HB':3, 'SC':4, 'Undefined':5})
df_tree['market_segment'] = df_tree['market_segment'].map({'Aviation':1, 'Complementary': 2, 'Corporate':3, 'Direct':4, 'Groups':5, 'Offline TA/TO':6, 'Online TA':7, 'Undefined': 8})
df_tree['distribution_channel'] = df_tree['distribution_channel'].map({'Corporate':1, 'Direct': 2, 'GDS':3, 'TA/TO':4, 'Undefined':5})
df_tree['reserved_room_type'] = df_tree['reserved_room_type'].map({'A':1, 'B': 2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'L':9, 'P':10})
df_tree['assigned_room_type'] = df_tree['assigned_room_type'].map({'A':1, 'B': 2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'L':9, 'P':10, 'I':11, 'K':12})
df_tree['deposit_type'] = df_tree['deposit_type'].map({'No Deposit':1, 'Non Refund': 2, 'Refundable':3})
df_tree['customer_type'] = df_tree['customer_type'].map({'Contract':1, 'Group': 2, 'Transient':3, 'Transient-Party':4})

In [252]:
df_tree['meal']

0        1
1        3
2        3
3        3
4        3
        ..
91526    1
91527    1
91528    1
91529    1
91530    1
Name: meal, Length: 91531, dtype: int64

In [253]:
df_tree.dtypes 

ID                                  int64
hotel                               int64
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                  int64
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                                int64
market_segment                      int64
distribution_channel                int64
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                  int64
assigned_room_type                  int64
booking_changes                     int64
deposit_type                        int64
days_in_waiting_list              

In [254]:
df_tree.isna().sum()

ID                                0
hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests   

In [258]:
train_tree = df_tree.loc[:74842, :]
val_tree = df_tree.loc[74843:, :]

X_train_tree = train_tree.drop(columns = ['is_canceled', 'adr'])
y_train_tree_isCan = train_tree['is_canceled']
y_train_tree_adr = train_tree['adr']
X_val_tree = val_tree.drop(columns = ['is_canceled', 'adr'])
y_val_tree_isCan = val_tree['is_canceled']
y_val_tree_adr = val_tree['adr']

In [241]:
def dateToWeekDay(df):
    year = df.arrival_date_year
    month = df.arrival_date_month
    day = df.arrival_date_day_of_month
    n = len(df)
    weekDays = [0 for i in range(n)]
    for i in range(n):
        weekDays[i] = date(year[i], month[i], day[i]).weekday()   
    # return 1~7
    df1 = copy.deepcopy(df)
    df1 = df1.rename(columns = {'arrival_date_year': 'year', 'arrival_date_month': 'month'})
    df1.insert(6,'weekDay', weekDays)
    df1 = pd.get_dummies(df1, columns = ['weekDay'])
    print(df1.columns)
    df1 = df1.drop(columns = ['weekDay_6']) # remove last one
    return df1;

In [147]:
df = dateToWeekDay(df_origin)
df_test = dateToWeekDay(df_test)

Index(['ID', 'hotel', 'is_canceled', 'lead_time', 'year', 'month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status_date', 'weekDay_0', 'weekDay_1', 'weekDay_2',
       'weekDay_3', 'weekDay_4', 'weekDay_5', 'weekDay_6'],
      dtype='object')
Index(['ID', 'hotel', 'lead_time', 'year', 'month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', '

In [148]:
df

,ID,hotel,is_canceled,lead_time,year,month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date,weekDay_0,weekDay_1,weekDay_2,weekDay_3,weekDay_4,weekDay_5
0,0,1,0,342,2015,7,27,1,0,0,...,-6.305161,0,0,2015-07-01,0,0,1,0,0,0
1,1,0,0,257,2015,7,27,1,0,2,...,75.052227,0,0,2015-07-03,0,0,1,0,0,0
2,2,0,0,257,2015,7,27,1,0,2,...,74.546401,0,0,2015-07-03,0,0,1,0,0,0
3,3,0,0,257,2015,7,27,1,0,2,...,76.376288,0,0,2015-07-03,0,0,1,0,0,0
4,4,0,0,257,2015,7,27,1,0,2,...,49.411647,0,0,2015-07-03,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,91526,1,1,19,2017,3,13,31,0,2,...,79.223571,0,1,2017-03-22,0,0,0,0,1,0
91527,91527,1,0,28,2017,3,13,31,0,2,...,-6.822102,0,0,2017-04-02,0,0,0,0,1,0
91528,91528,1,0,2,2017,3,13,31,0,1,...,90.814554,0,2,2017-04-01,0,0,0,0,1,0
91529,91529,1,0,30,2017,3,13,31,3,7,...,38.135565,0,1,2017-04-10,0,0,0,0,1,0


In [149]:
df_test

,ID,hotel,lead_time,year,month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,weekDay_0,weekDay_1,weekDay_2,weekDay_3,weekDay_4,weekDay_5
0,91531,0,75,2017,4,13,1,2,5,2,...,0,Transient,0,1,0,0,0,0,0,1
1,91532,0,208,2017,4,13,1,4,10,2,...,0,Transient,0,1,0,0,0,0,0,1
2,91533,1,12,2017,4,13,1,2,5,2,...,0,Contract,0,1,0,0,0,0,0,1
3,91534,0,76,2017,4,13,1,2,5,3,...,0,Transient,0,0,0,0,0,0,0,1
4,91535,0,9,2017,4,13,1,2,4,2,...,0,Transient,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27854,119385,1,108,2017,8,35,31,2,5,2,...,0,Transient,0,1,0,0,0,1,0,0
27855,119386,1,194,2017,8,35,31,2,5,2,...,0,Transient,1,1,0,0,0,1,0,0
27856,119387,1,17,2017,8,35,31,0,3,2,...,0,Transient,0,2,0,0,0,1,0,0
27857,119388,1,191,2017,8,35,31,2,5,2,...,0,Contract,0,0,0,0,0,1,0,0


In [150]:
df.isna().sum()

ID                                    0
hotel                                 0
is_canceled                           0
lead_time                             0
year                                  0
month                                 0
arrival_date_week_number              0
arrival_date_day_of_month             0
stays_in_weekend_nights               0
stays_in_week_nights                  0
adults                                0
children                              4
babies                                0
meal                                  0
country                             468
market_segment                        0
distribution_channel                  0
is_repeated_guest                     0
previous_cancellations                0
previous_bookings_not_canceled        0
reserved_room_type                    0
assigned_room_type                    0
booking_changes                       0
deposit_type                          0
agent                             13217


"children" has few NAs, while "country", "agent", and "company" have a lot of NA values. So we will discard "country", "agent", "company" attributes, and fill in median in NA of "children".

In [151]:
df_test.isna().sum()

ID                                    0
hotel                                 0
lead_time                             0
year                                  0
month                                 0
arrival_date_week_number              0
arrival_date_day_of_month             0
stays_in_weekend_nights               0
stays_in_week_nights                  0
adults                                0
children                              0
babies                                0
meal                                  0
country                              20
market_segment                        0
distribution_channel                  0
is_repeated_guest                     0
previous_cancellations                0
previous_bookings_not_canceled        0
reserved_room_type                    0
assigned_room_type                    0
booking_changes                       0
deposit_type                          0
agent                              3123
company                           26676


In [152]:
df = df.drop(columns = ['year', 'country', 'agent', 'company', 'arrival_date_week_number', 'arrival_date_day_of_month'])
df_test = df_test.drop(columns = ['year', 'country', 'agent', 'company', 'arrival_date_week_number', 'arrival_date_day_of_month'])

In [15]:
df = df.fillna(df.children.median())

In [16]:
df.columns

Index(['ID', 'hotel', 'is_canceled', 'lead_time', 'month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'meal', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status_date', 'weekDay_0', 'weekDay_1', 'weekDay_2',
       'weekDay_3', 'weekDay_4', 'weekDay_5'],
      dtype='object')

In [153]:
df_test.columns

Index(['ID', 'hotel', 'lead_time', 'month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'market_segment', 'distribution_channel', 'is_repeated_guest',
       'previous_cancellations', 'previous_bookings_not_canceled',
       'reserved_room_type', 'assigned_room_type', 'booking_changes',
       'deposit_type', 'days_in_waiting_list', 'customer_type',
       'required_car_parking_spaces', 'total_of_special_requests', 'weekDay_0',
       'weekDay_1', 'weekDay_2', 'weekDay_3', 'weekDay_4', 'weekDay_5'],
      dtype='object')

In [166]:
# Creating dummy variables:
df_dum = pd.get_dummies(df, columns=['month', 'meal', 'market_segment', 'distribution_channel', \
                                 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type'])
df_test_dum = pd.get_dummies(df_test, columns=['month', 'meal', 'market_segment', 'distribution_channel', \
                                 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type'])
print(df_dum.columns)
print(df_test_dum.columns)

Index(['ID', 'hotel', 'is_canceled', 'lead_time', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes',
       'days_in_waiting_list', 'adr', 'required_car_parking_spaces',
       'total_of_special_requests', 'reservation_status_date', 'weekDay_0',
       'weekDay_1', 'weekDay_2', 'weekDay_3', 'weekDay_4', 'weekDay_5',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'meal_BB', 'meal_FB', 'meal_HB', 'meal_SC', 'meal_Undefined',
       'market_segment_Aviation', 'market_segment_Complementary',
       'market_segment_Corporate', 'market_segment_Direct',
       'market_segment_Groups', 'market_segment_Offline TA/TO',
       'market_segment_Online TA', 'market_segment_Undefined',
       'distribution_channel_Corporate', 'distribution_cha

In [167]:
# drop the last column of each dummy category
df_dum = df_dum.drop(columns = ['reservation_status_date', 'weekDay_5', 'month_8', 'meal_Undefined', 'market_segment_Undefined', 'market_segment_Undefined', 'distribution_channel_Undefined', 'assigned_room_type_P', 'deposit_type_Refundable', 'customer_type_Transient-Party'])

In [168]:
df_test_dum = df_test_dum.drop(columns = [''])

NameError: name 'columns' is not defined

In [160]:
df_dum

,ID,hotel,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,...,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,deposit_type_No Deposit,deposit_type_Non Refund,customer_type_Contract,customer_type_Group,customer_type_Transient
0,0,1,0,342,0,0,2,0.0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,1,0,0,257,0,2,1,0.0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,2,0,0,257,0,2,2,0.0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,3,0,0,257,0,2,2,0.0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,4,0,0,257,0,2,2,0.0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,91526,1,1,19,0,2,2,2.0,0,0,...,0,0,0,0,0,1,0,0,0,1
91527,91527,1,0,28,0,2,2,0.0,0,0,...,0,0,0,0,0,1,0,0,0,1
91528,91528,1,0,2,0,1,2,0.0,0,0,...,0,0,0,0,0,1,0,0,0,1
91529,91529,1,0,30,3,7,2,0.0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [20]:
df_dum.columns

Index(['ID', 'hotel', 'is_canceled', 'lead_time', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes',
       'days_in_waiting_list', 'adr', 'required_car_parking_spaces',
       'total_of_special_requests', 'weekDay_0', 'weekDay_1', 'weekDay_2',
       'weekDay_3', 'weekDay_4', 'weekDay_5', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'meal_BB', 'meal_FB', 'meal_HB', 'meal_SC',
       'market_segment_Aviation', 'market_segment_Complementary',
       'market_segment_Corporate', 'market_segment_Direct',
       'market_segment_Groups', 'market_segment_Offline TA/TO',
       'market_segment_Online TA', 'distribution_channel_Corporate',
       'distribution_channel_Direct', 'distribution_channel_GDS',
       'distribution_channel_TA/TO', 'reserved_roo

## Basic analysis

#### Target: "is_canceled"

Steps:

1. Predict "is_canceled" by attributes in each tuple

2. Multiply predictions by "ADR"

3. Group by date

4. Predict classification of the daily revenue to the labels on $y$.


In [21]:
df_dum.describe()

,ID,hotel,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,...,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,deposit_type_No Deposit,deposit_type_Non Refund,customer_type_Contract,customer_type_Group,customer_type_Transient
count,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,...,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000
mean,45765.000000,0.342299,0.357912,96.286843,0.899291,2.448438,1.835761,0.090964,0.008369,0.033890,...,0.020102,0.005714,0.003256,0.002546,0.000011,0.869957,0.128568,0.037834,0.004873,0.723646
std,26422.868082,0.474482,0.479389,105.454726,1.004555,1.935080,0.605828,0.373909,0.101749,0.180947,...,0.140352,0.075375,0.056966,0.050390,0.003305,0.336353,0.334724,0.190796,0.069635,0.447197
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22882.500000,0.000000,0.000000,15.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,45765.000000,0.000000,0.000000,58.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,68647.500000,1.000000,1.000000,145.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,91530.000000,1.000000,1.000000,737.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### Building X and y

#### Splitting train/validation
(isCan/adr)

Split data at index 74843

:74842 -> train

74843: -> val

In [56]:
train = df_dum.loc[:74842, :]
val = df_dum.loc[74843:, :]
df_train = df_origin.loc[:74842, :] # for revenue to label
df_val = df_origin.loc[74843:, :] # for revenue to label

X_train = train.drop(columns = ['is_canceled', 'adr'])
y_train_isCan = train['is_canceled']
y_train_adr = train['adr']
X_val = val.drop(columns = ['is_canceled', 'adr'])
y_val_isCan = val['is_canceled']
y_val_adr = val['adr']
# ~2016/11

In [57]:
X_train

,ID,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,...,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,deposit_type_No Deposit,deposit_type_Non Refund,customer_type_Contract,customer_type_Group,customer_type_Transient
0,0,1,342,0,0,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,1,0,257,0,2,1,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,2,0,257,0,2,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,3,0,257,0,2,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,4,0,257,0,2,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74838,74838,1,1,0,3,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
74839,74839,1,1,0,2,1,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
74840,74840,0,138,0,3,1,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
74841,74841,0,12,0,3,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [58]:
X_val

,ID,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,...,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,deposit_type_No Deposit,deposit_type_Non Refund,customer_type_Contract,customer_type_Group,customer_type_Transient
74843,74843,0,290,1,3,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
74844,74844,0,510,0,2,2,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
74845,74845,0,510,0,2,2,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
74846,74846,0,510,0,2,2,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
74847,74847,0,510,0,2,2,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,91526,1,19,0,2,2,2.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
91527,91527,1,28,0,2,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
91528,91528,1,2,0,1,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
91529,91529,1,30,3,7,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [59]:
y_train_isCan.value_counts(normalize=True)

0    0.638163
1    0.361837
Name: is_canceled, dtype: float64

The cancel rate is around 35%.

In [60]:
y_train_adr.value_counts()

0.000000      75
47.000000      5
31.000000      5
42.000000      5
16.000000      4
              ..
139.827646     1
79.667069      1
115.188575     1
151.865524     1
50.842307      1
Name: adr, Length: 74732, dtype: int64

### Computing accuracy 

In [262]:
from sklearn.metrics import accuracy_score
def computeAccuracy(X, y, y_result): #x: test revenue, y: test label
    print("Accuracy =", accuracy_score(y, y_result))
    print("Number of mislabeled points out of a total %d points : %d"% (X.shape[0],(y != y_result).sum()))
    print("E_val = ", (y != y_result).sum() / X.shape[0])
    
from sklearn.metrics import r2_score
def calculateRsquare(X, y, y_result): #x: test revenue, y: test label
    print("R square =", r2_score(y, y_result))

## Predict is_can

### KNN

In [54]:
def trainKNN(X_train_df, y_train, k, X_val_df, y_val):
    print("Pre-processing...")
#     X_train_feat = X_train_df[['ID', 'lead_time', 'previous_cancellations', \
#                             'previous_bookings_not_canceled', 'booking_changes', \
#                             'days_in_waiting_list', 'required_car_parking_spaces', \
#                             'total_of_special_requests', 'market_segment_Groups', 'deposit_type_Non Refund']]
#     X_train_feat = X_train_df[['ID', 'lead_time', 'previous_cancellations', 'days_in_waiting_list', 'deposit_type_Non Refund']]
#     X_train_feat = X_train_feat.to_numpy()
    X_train_feat = X_train_df.to_numpy()
    knn = KNeighborsClassifier(n_neighbors = k)
    print("Fitting...")
    knn_best = knn.fit(X_train_feat, y_train)
#     X_val_feat = X_val_df[['ID', 'lead_time', 'previous_cancellations', \
#                             'previous_bookings_not_canceled', 'booking_changes', \
#                             'days_in_waiting_list', 'required_car_parking_spaces', \
#                             'total_of_special_requests', 'market_segment_Groups', 'deposit_type_Non Refund']]
#     X_val_feat = X_val_df[['distribution_channel_GDS', 'reserved_room_type_B', 'reserved_room_type_L', 'assigned_room_type_B', 'assigned_room_type_L']]
#     X_val_feat = X_val_feat.to_numpy()
    X_val_feat = X_val_df.to_numpy()
    print("Predicting...")
    y_pred = knn_best.predict(X_val_feat)
    print("Calculating accuracy...")
    sc = knn_best.score(X_val_feat, y_val, sample_weight=None)
    return y_pred, sc;

In [55]:
y_pred, sc = train_KNN_isCan(X_train, y_train_isCan, 5, X_val, y_val_isCan)

NameError: name 'train_KNN_isCan' is not defined

In [185]:
sc

0.6679050814956855

In [187]:
computeAccuracy(X_val, y_val_isCan, y_pred)

Accuracy = 0.6679050814956855
Number of mislabeled points out of a total 16688 points : 5542
E_val =  0.3320949185043145


### Desicion Stump

In [315]:
from sklearn import tree

def trainDecisionStump(X_train, y_train, X_val):
    clf = tree.DecisionTreeClassifier().fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

def trainDecisionStumpRegressor(X_train, y_train, X_val):
    clf = tree.DecisionTreeRegressor(max_depth=10).fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

In [81]:
y_result = trainDecisionStump(X_train, y_train_isCan, X_val)

In [82]:
computeAccuracy(X_val, y_val_isCan, y_result)

Accuracy = 0.6718600191754555
Number of mislabeled points out of a total 16688 points : 5476
E_val =  0.3281399808245446


#### tree isCan

In [260]:
y_result = trainDecisionStump(X_train_tree, y_train_tree_isCan, X_val_tree)

In [263]:
computeAccuracy(X_val_tree, y_val_tree_isCan, y_result)

Accuracy = 0.652984180249281
Number of mislabeled points out of a total 16688 points : 5791
E_val =  0.3470158197507191


#### tree adr

In [316]:
y_result = trainDecisionStumpRegressor(X_train_tree, y_train_tree_adr, X_val_tree)

In [317]:
calculateRsquare(X_val_tree, y_val_tree_adr, y_result)

R square = 0.4756839274931203


### Logistic Regression

使用 L2 regularization, 搭配上'sag'solver

In [117]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


### solver: lbfgs, newton-cg, sag
def trainLogisticRegression(X_train, y_train, X_val):
    clf = LogisticRegression(random_state=0, penalty='l2', solver='newton-cg', multi_class='multinomial').fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

In [118]:
y_result = trainLogisticRegression(X_train, y_train_isCan, X_val)

C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [119]:
computeAccuracy(X_val, y_val_isCan, y_result)

Accuracy = 0.7870925215723873
Number of mislabeled points out of a total 16688 points : 3553
E_val =  0.21290747842761265


### SVM

In [ ]:
import numpy as np
import sys
sys.path.insert(1, '.')
from svmutil import *

In [ ]:
def toSVMdata(X, y): 
    N, d = X_train.shape
    X_svm = []
    for n in range(N):
        X_svm.append({})
        for i in range(d):
            X_svm[n].update({i: X_train[n,i]}) 
    y_svm = list(y)
    return X_svm, y_svm

In [ ]:
def SVM (Xtrain, ytrain, Xval, yval):
    bestaccu = 0
    for i in range(-2, 3):
        c = pow(10, i)
        # turn into SVM data form
        Xval, yval = toSVMdata(Xval, yval)
        Xtrain, ytrain = toSVMdata(Xtrain, ytrain)
        # train and prediction
        model = svm_train(ytrain, Xtrain, '-s 0 -t 0 -c {C}'.format(C = c))
        print("with C =", c, ",", end = "")
        p_label, p_acc, p_val = svm_predict(yval, Xval, model)
        if (p_acc[0] > bestaccu):
            bestaccu = p_acc[0]
    bestaccu /= 100
    return bestaccu

In [ ]:
a = X_train.to_numpy()
b = y_train_isCan.to_numpy()
print(type(a))
print(type(b))
X_train_svm, y_train_svm = toSVMdata(X_train.to_numpy(), y_train_isCan.to_numpy())
X_val_svm, y_val_svm = toSVMdata(X_val, y_val_isCan)
accuracy = SVM(X_train_svm, y_train_svm, X_val_svm, y_val_svm)
print("E_val =", 1 - accuracy)

### GussianNB

In [83]:
from sklearn.naive_bayes import GaussianNB

def trainGussianNB(X_train, y_train, X_val):
    clf = GaussianNB().fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

In [84]:
y_result = trainGussianNB(X_train, y_train_isCan, X_val)

In [85]:
computeAccuracy(X_val, y_val_isCan, y_result)

Accuracy = 0.7596476510067114
Number of mislabeled points out of a total 16688 points : 4011
E_val =  0.24035234899328858


## Predict adr

### Neural Network

In [117]:
from sklearn.neural_network import MLPClassifier

def trainNeural(X_train, y_train, X_val):
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1).fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

In [131]:
# y_result = trainNeural(X_train, y_train_adr, X_val)

In [ ]:
# calculateRsquare(X_val, y_val_adr, y_result)

### Linear Regression

In [70]:
from sklearn.linear_model import LinearRegression

def trainLinearRegression(X_train_adr, y_train_adr, X_val_adr):
    clf = LinearRegression().fit(X_train_adr, y_train_adr)
    y_result = clf.predict(X_val_adr)
    return(y_result)

In [71]:
y_result = trainLinearRegression(X_train, y_train_adr, X_val)

In [72]:
calculateRsquare(X_val, y_val_adr, y_result)

R square = 0.44109509968084737


### AdaboostClassifier (for is_canceled)

from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification


def AdaboostClassifier(x, y):
    clf = AdaBoostClassifier(n_estimators=100, random_state=0, algorithm='SAMME')
    clf.fit(x, y) 
    y_result = clf.predict(x)
    return(y_result)
    

### Calculate predicted daily revenue

In [106]:
def calculateDateRevenue(df):
    df1 = df[["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month", "adr", "is_canceled", "stays_in_weekend_nights", "stays_in_week_nights"]]
    df1["revenue"] = (1 - df1["is_canceled"]) * df1["adr"] * (df1["stays_in_week_nights"] + df1["stays_in_weekend_nights"])
    df2 = df1[["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month", "revenue"]]
    df3 = df2.groupby(by=["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"]).sum()
    x = df3.to_numpy()
    return(x) # one column: revenue

## Revenue to label 

In [107]:
df_train.columns

Index(['ID', 'hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status_date'],
      dtype='object')

In [108]:
X_train_revenue = calculateDateRevenue(df_train)
# print(len(X_train_revenue))
X_val_revenue = calculateDateRevenue(df_val)
# print(len(X_val_revenue))

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [109]:
label["ID"] = label.index
label_train = label.loc[:518, :]
label_val = label.loc[519:, :]

In [110]:
y_train_label = label_train["label"].to_numpy()
# print(len(y_train_label))
y_val_label = label_val["label"].to_numpy()
# print(len(y_val_label))

for i in range(len(X_train_revenue)):
    print(X_train_revenue[i], y_train_label[i])

### KNN

In [204]:
y_result = trainKNN(X_train_revenue, y_train_label, 5, X_val_revenue, y_val_label)

Pre-processing...


AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [116]:
computeAccuracy(X_val_revenue, y_val_revenue, y_result)

### Desicion Stump 

In [111]:
y_result = trainDecisionStump(X_train_revenue, y_train_label, X_val_revenue)

In [112]:
computeAccuracy(X_val_revenue, y_val_label, y_result)

Accuracy = 0.9917355371900827
Number of mislabeled points out of a total 121 points : 1
E_val =  0.008264462809917356


### Logistic Regression

In [113]:
y_result = trainLogisticRegression(X_train_revenue, y_train_label, X_val_revenue)

C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [114]:
computeAccuracy(X_val_revenue, y_val_label, y_result)

Accuracy = 0.9256198347107438
Number of mislabeled points out of a total 121 points : 9
E_val =  0.0743801652892562


### SVM

In [105]:
y_result = trainSVM(X_train_revenue, y_train_label, X_val_revenue)

In [78]:
computeAccuracy(X_val_revenue, y_val_label, y_result)

Accuracy = 0.38016528925619836
Number of mislabeled points out of a total 121 points : 75
E_val =  0.6198347107438017


### GuassianNB

In [115]:
y_result = trainGussianNB(X_train_revenue, y_train_label, X_val_revenue)

In [116]:
computeAccuracy(X_val_revenue, y_val_label, y_result)

Accuracy = 0.9504132231404959
Number of mislabeled points out of a total 121 points : 6
E_val =  0.049586776859504134


## Test

In [ ]:
isCan = trainLogisticRegression(X_train, y_train_isCan, X_test)
